# Tutorial ACS_UPB_LAB1: Running Sumo Simulations

__Credits: most of the credits for this ipynb goes to https://github.com/flow-project/flow/tree/master/tutorials__

This tutorial walks through the process of running non-RL traffic simulations in Flow. Simulations of this form act as non-autonomous baselines and depict the behavior of human dynamics on a network. Similar simulations may also be used to evaluate the performance of hand-designed controllers on a network. This tutorial focuses primarily on the former use case, while an example of the latter may be found in `exercise07_controllers.ipynb`.

In this exercise, we simulate a initially perturbed single lane ring road. We witness in simulation that as time advances the initially perturbations do not dissipate, but instead propagates and expands until vehicles are forced to periodically stop and accelerate. For more information on this behavior, we refer the reader to the following article [1].

## 1.1 Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *network*, and an *environment*. Networks describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc. in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act the primary interface between the reinforcement learning algorithm and the network. Moreover, custom environments may be used to modify the dynamical features of an network.

## 1.2 Setting up the environment of current lab (ENV1)
Load configurations for lab 1.

## 2. Setting up a Network
Flow contains a plethora of pre-designed networks used to replicate highways, intersections, and merges in both closed and open settings. All these networks are located in flow/networks. In order to recreate a ring road network, we begin by importing the network `RingNetwork`.

In [1]:
from flow.envs.nemodrive_lab import ENV1 as ENV

# from flow.networks.figure_eight import FigureEightNetwork
network_name = ENV["NETWORK"]
print(network_name.__name__)

FigureEightNetwork


This network, as well as all other networks in Flow, is parametrized by the following arguments: 
* name
* vehicles
* net_params
* initial_config
* traffic_lights

These parameters allow a single network to be recycled for a multitude of different network settings. For example, `RingNetwork` may be used to create ring roads of variable length with a variable number of lanes and vehicles.

### 2.1 Name
The `name` argument is a string variable depicting the name of the network. This has no effect on the type of network created.

In [2]:
name = network_name.__name__

### 2.2 VehicleParams
The `VehicleParams` class stores state information on all vehicles in the network. This class is used to identify the dynamical behavior of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various get methods within this class.

The initial configuration of this class describes the number of vehicles in the network at the start of every simulation, as well as the properties of these vehicles. We begin by creating an empty `VehicleParams` object.

In [3]:
vehicles = ENV["VEHICLES"]()

# code in get_vehicles 
# from flow.core.params import VehicleParams

# vehicles = VehicleParams()

Once this object is created, vehicles may be introduced using the `add` method. This method specifies the types and quantities of vehicles at the start of a simulation rollout. For a description of the various arguements associated with the `add` method, we refer the reader to the following documentation ([VehicleParams.add](https://flow.readthedocs.io/en/latest/flow.core.html?highlight=vehicleparam#flow.core.params.VehicleParams)).

When adding vehicles, their dynamical behaviors may be specified either by the simulator (default), or by user-generated models. For longitudinal (acceleration) dynamics, several prominent car-following models are implemented in Flow. For this example, the acceleration behavior of all vehicles will be defined by the Intelligent Driver Model (IDM) [2].

In [4]:
# code in get_vehicles 
# from flow.controllers.car_following_models import IDMController

Another controller we define is for the vehicle's routing behavior. For closed network where the route for any vehicle is repeated, the `ContinuousRouter` controller is used to perpetually reroute all vehicles to the initial set route.

In [5]:
# code in get_vehicles 
# from flow.controllers.routing_controllers import ContinuousRouter

Finally, we add 22 vehicles of type "human" with the above acceleration and routing behavior into the `Vehicles` class.

In [6]:
# (E.g. code in get_vehicles)
# vehicles.add("human",
#              acceleration_controller=(IDMController, {}),
#              routing_controller=(ContinuousRouter, {}),
#              num_vehicles=22)

### 2.3 NetParams

`NetParams` are network-specific parameters used to define the shape and properties of a network. Unlike most other parameters, `NetParams` may vary drastically depending on the specific network configuration, and accordingly most of its parameters are stored in `additional_params`. In order to determine which `additional_params` variables may be needed for a specific network, we refer to the `ADDITIONAL_NET_PARAMS` variable located in the network file.

In [7]:
# from flow.networks.ring import ADDITIONAL_NET_PARAMS

ADDITIONAL_NET_PARAMS = ENV["ADDITIONAL_NET_PARAMS"]

print(ADDITIONAL_NET_PARAMS)

{'radius_ring': 60, 'lanes': 2, 'speed_limit': 30, 'resolution': 40}


Importing the `ADDITIONAL_NET_PARAMS` dict from the ring road network, we see that the required parameters are:

* **length**: length of the ring road
* **lanes**: number of lanes
* **speed**: speed limit for all edges
* **resolution**: resolution of the curves on the ring. Setting this value to 1 converts the ring to a diamond.


At times, other inputs may be needed from `NetParams` to recreate proper network features/behavior. These requirements can be founded in the network's documentation. For the ring road, no attributes are needed aside from the `additional_params` terms. Furthermore, for this exercise, we use the network's default parameters when creating the `NetParams` object.

In [8]:
from flow.core.params import NetParams

net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

### 2.4 InitialConfig

`InitialConfig` specifies parameters that affect the positioning of vehicle in the network at the start of a simulation. These parameters can be used to limit the edges and number of lanes vehicles originally occupy, and provide a means of adding randomness to the starting positions of vehicles. In order to introduce a small initial disturbance to the system of vehicles in the network, we set the `perturbation` term in `InitialConfig` to 1m.

In [9]:
from flow.core.params import InitialConfig
initial_config_param = ENV["INITIAL_CONFIG_PARAMS"]
print(initial_config_param)

initial_config = InitialConfig(**initial_config_param)

{'spacing': 'random', 'perturbation': 50}


### 2.5 TrafficLightParams

`TrafficLightParams` are used to describe the positions and types of traffic lights in the network. These inputs are outside the scope of this tutorial, and instead are covered in `exercise06_traffic_lights.ipynb`. For our example, we create an empty `TrafficLightParams` object, thereby ensuring that none are placed on any nodes.

In [10]:
from flow.core.params import TrafficLightParams

traffic_lights = TrafficLightParams()

## 3. Setting up an Environment

Several envionrments in Flow exist to train autonomous agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. These environments are often network or task specific; however, some can be deployed on an ambiguous set of networks as well. One such environment, `AccelEnv`, may be used to train a variable number of vehicles in a fully observable network with a *static* number of vehicles.

In [11]:
# from flow.envs.nemodrive_lab.env1_lab import LaneChangeAccelEnv1
env_name = ENV["ENVIRONMENT"]
print(env_name)

<class 'flow.envs.nemodrive_lab.env1_lab.LaneChangeAccelEnv1'>


Although we will not be training any autonomous agents in this exercise, the use of an environment allows us to view the cumulative reward simulation rollouts receive in the absence of autonomy.

Envrionments in Flow are parametrized by three components:
* `EnvParams`
* `SumoParams`
* `Network`

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables. These variables include the length a simulation step (in seconds) and whether to render the GUI when running the experiment. For this example, we consider a simulation step length of 0.1s and activate the GUI.

Another useful parameter is `emission_path`, which is used to specify the path where the emissions output will be generated. They contain a lot of information about the simulation, for instance the position and speed of each car at each time step. If you do not specify any emission path, the emission file will not be generated. More on this in Section 5.

In [12]:
from flow.core.params import SumoParams

sumo_params = SumoParams(sim_step=0.1, render=True, emission_path='data', restart_instance=True)

### 3.2 EnvParams

`EnvParams` specify environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. Much like `NetParams`, the attributes associated with this parameter are mostly environment specific, and can be found in the environment's `ADDITIONAL_ENV_PARAMS` dictionary.

In [13]:
# from flow.envs.nemodrive_lab.env1_lab import ADDITIONAL_ENV1_PARAMS
ADDITIONAL_ENV_PARAMS = ENV["ADDITIONAL_ENV_PARAMS"]

print(ADDITIONAL_ENV_PARAMS)

{'max_accel': 3, 'max_decel': 3, 'lane_change_duration': 5, 'target_velocity': 10, 'sort_vehicles': False, 'forward_progress_gain': 0.1, 'collision_reward': -1, 'lane_change_reward': -0.1, 'frontal_collision_distance': 2.0, 'lateral_collision_distance': 3.0}


Importing the `ADDITIONAL_ENV_PARAMS` variable, we see that it consists of only one entry, "target_velocity", which is used when computing the reward function associated with the environment. We use this default value when generating the `EnvParams` object.

In [14]:
from flow.core.params import EnvParams

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=ENV["HORIZON"])

## 4. Setting up and Running the Experiment
Once the inputs to the network and environment classes are ready, we are ready to set up a `Experiment` object.

In [15]:
from flow.core.experiment import Experiment

These objects may be used to simulate rollouts in the absence of reinforcement learning agents, as well as acquire behaviors and rewards that may be used as a baseline with which to compare the performance of the learning agent. In this case, we choose to run our experiment for one rollout consisting of 3000 steps (300 s).

**Note**: When executing the below code, remeber to click on the    <img style="display:inline;" src="img/play_button.png"> Play button after the GUI is rendered.

In [16]:
# create the network object
network = network_name(name="ring_example",
                       vehicles=vehicles,
                       net_params=net_params,
                       initial_config=initial_config,
                       traffic_lights=traffic_lights)



In [ ]:
# create the environment object
sumo_params.render = False
env = env_name(env_params, sumo_params, network)

# create the experiment object
exp = Experiment(env)
_ = exp.run(1, 3000, convert_to_csv=True)


Run still agent.

In [ ]:
sumo_params.render = False
env = env_name(env_params, sumo_params, network)

# create the experiment object
exp = Experiment(env)

rl_actions = lambda state: [0, 0]

_ = exp.run(1, 3000, convert_to_csv=True, rl_actions=rl_actions)

Run random agent.

Use __FullExperiment__ to test agent that expects _state, reward, done, info_.

In [ ]:
from flow.core.experiment_with_reward import FullExperiment
import numpy as np

class RandomAgent():
    def __init__(self, env):
        self.action_space = env.action_space
        self.max_decel = env.env_params.additional_params["max_decel"]
        self.max_accel = env.env_params.additional_params["max_accel"]
        self.change_lane_step_freq = 10
        self.num_steps = 0
        
    def act(self, state, reward, done, info):
        self.num_steps += 1
        d = 0
        if self.num_steps % self.change_lane_step_freq == 0:
            d = np.random.randint(3) - 1

        acc = np.random.uniform(-self.max_decel, self.max_accel)
        action =  np.array([acc, d])

        yield action

sumo_params.render = False
env = env_name(env_params, sumo_params, network)

exp = FullExperiment(env)

agent = RandomAgent(env)

_ = exp.run(10, 3000, convert_to_csv=True, rl_actions=agent.act)


Feel free to experiment with all these problems and more!

## Bibliography
[1] Sugiyama, Yuki, et al. "Traffic jams without bottlenecks—experimental evidence for the physical mechanism of the formation of a jam." New journal of physics 10.3 (2008): 033001.

[2] Treiber, Martin, Ansgar Hennecke, and Dirk Helbing. "Congested traffic states in empirical observations and microscopic simulations." Physical review E 62.2 (2000): 1805.

## 5 Setting up Flow Parameters

RLlib experiments both generate a `params.json` file for each experiment run. For RLlib experiments, the parameters defining the Flow network and environment must be stored as well. As such, in this section we define the dictionary `flow_params`, which contains the variables required by the utility function `make_create_env`. `make_create_env` is a higher-order function which returns a function `create_env` that initializes a Gym environment corresponding to the Flow network specified.

In [17]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
sumo_params.render = False
sumo_params.print_warnings=False
flow_params = dict(
    # name of the experiment
    exp_tag=name,
    # name of the flow environment the experiment is running on
    env_name=env_name,
    # name of the network class the experiment uses
    network=network_name,
    # simulator that is used by the experiment
    simulator='traci',
    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=sumo_params,
    # environment related parameters (see flow.core.params.EnvParams)
    env=env_params,
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,
    # vehicles to be placed in the network at the start of a rollout 
    # (see flow.core.vehicles.Vehicles)
    veh=vehicles,
    # (optional) parameters affecting the positioning of vehicles upon 
    # initialization/reset (see flow.core.params.InitialConfig)
    initial=initial_config
)

## 4 Running RL experiments in Ray

### 4.1 Import 

First, we must import modules required to run experiments in Ray. The `json` package is required to store the Flow experiment parameters in the `params.json` file, as is `FlowParamsEncoder`. Ray-related imports are required: the PPO algorithm agent, `ray.tune`'s experiment runner, and environment helper methods `register_env` and `make_create_env`.

In [18]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

/home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/andrei/anaconda3/envs/flow/lib/python3.6/site-

### 4.2 Initializing Ray
Here, we initialize Ray and experiment-based constant variables specifying parallelism in the experiment as well as experiment batch size in terms of number of rollouts.

In [19]:
# number of parallel workers
N_CPUS = 8
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(num_cpus=N_CPUS)

2019-11-29 12:28:31,294	INFO resource_spec.py:205 -- Starting Ray with 1.71 GiB memory available for workers and up to 0.87 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '172.19.3.168',
 'redis_address': '172.19.3.168:38662',
 'object_store_address': '/tmp/ray/session_2019-11-29_12-28-31_293489_18475/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-29_12-28-31_293489_18475/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-11-29_12-28-31_293489_18475'}

### 4.3 Configuration and Setup
Here, we copy and modify the default configuration for the [PPO algorithm](https://arxiv.org/abs/1707.06347). The agent has the number of parallel workers specified, a batch size corresponding to `N_ROLLOUTS` rollouts (each of which has length `HORIZON` steps), a discount rate $\gamma$ of 0.999, two hidden layers of size 16, uses Generalized Advantage Estimation, $\lambda$ of 0.97, and other parameters as set below.

Once `config` contains the desired parameters, a JSON string corresponding to the `flow_params` specified in section 3 is generated. The `FlowParamsEncoder` maps objects to string representations so that the experiment can be reproduced later. That string representation is stored within the `env_config` section of the `config` dictionary. Later, `config` is written out to the file `params.json`. 

Next, we call `make_create_env` and pass in the `flow_params` to return a function we can use to register our Flow environment with Gym. 

In [20]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"
HORIZON = 100

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 500  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

### 4.4 Running Experiments

Here, we use the `run_experiments` function from `ray.tune`. The function takes a dictionary with one key, a name corresponding to the experiment, and one value, itself a dictionary containing parameters for training.

In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 500,  # number of iterations to stop after
        },
    },
})

2019-11-29 12:28:34,261	WARNING logger.py:343 -- Could not instantiate tf2_compat_logger: module 'tensorflow.compat' has no attribute 'v1'.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs, 0.0/1.71 GiB heap, 0.0/0.59 GiB objects
Memory usage on this node: 12.2/15.6 GiB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 8/8 CPUs, 0/1 GPUs, 0.0/1.71 GiB heap, 0.0/0.59 GiB objects
Memory usage on this node: 12.3/15.6 GiB
Result logdir: /home/andrei/ray_results/FigureEightNetwork
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_LaneChangeAccelEnv1-v0_0:	RUNNING

(pid=18531) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=18531)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=18531) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synony

(pid=18528) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=18528)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=18528) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=18528)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=18528) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=18528)   _np_qint16 = np.dtype([("qint

(pid=18530) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=18530)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=18530) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=18530)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=18530) /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=18530)   _np_qint16 = np.dtype([("qint

(pid=18529) WARNING:tensorflow:From /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/ray/rllib/models/tf/tf_action_dist.py:73: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
(pid=18529) Instructions for updating:
(pid=18529) keep_dims is deprecated, use keepdims instead
(pid=18529) WARNING:tensorflow:From /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/ray/rllib/models/tf/tf_action_dist.py:73: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
(pid=18529) Instructions for updating:
(pid=18529) keep_dims is deprecated, use keepdims instead
(pid=18525) WARNING:tensorflow:From /home/andrei/anaconda3/envs/flow/lib/python3.6/site-packages/ray/rllib/models/tf/tf_action_dist.py:68: calling reduce_max (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
(pid=185

...

(pid=31376) /home/andrei/workspace/flow_nemodrive/flow/core/kernel/vehicle/traci.py:951: UserWarning: API change now handles duration as floating point seconds
(pid=31376)   veh_id, int(target_lane), 100000)
(pid=31376) Warning: Vehicle 'human_9_0' performs emergency braking with decel=-23.28 wished=4.50 severity=4.17, time=2.70.
(pid=31375) /home/andrei/workspace/flow_nemodrive/flow/core/kernel/vehicle/traci.py:951: UserWarning: API change now handles duration as floating point seconds
(pid=31375)   veh_id, int(target_lane), 100000)
(pid=31376) Warning: Vehicle 'human_9_0' performs emergency braking with decel=-21.04 wished=4.50 severity=3.68, time=5.60.
(pid=31380) /home/andrei/workspace/flow_nemodrive/flow/core/kernel/vehicle/traci.py:951: UserWarning: API change now handles duration as floating point seconds
(pid=31380)   veh_id, int(target_lane), 100000)
(pid=31376) Warning: Vehicle 'human_9_0' performs emergency braking with decel=-15.78 wished=4.50 severity=2.51, time=8.50.
(pid

(pid=31378) Warning: Vehicle 'human_2_0' performs emergency braking with decel=-11.33 wished=4.50 severity=1.52, time=4.80.
(pid=31378) Warning: Vehicle 'human_2_0' performs emergency braking with decel=-13.65 wished=4.50 severity=2.03, time=4.90.
(pid=31375) Warning: Vehicle 'human_13_0' performs emergency braking with decel=-13.98 wished=4.50 severity=2.11, time=5.60.
(pid=31378) Warning: Vehicle 'human_13_0' performs emergency braking with decel=-49.46 wished=4.50 severity=9.99, time=6.60.
(pid=31375) Warning: Vehicle 'human_12_0' performs emergency braking with decel=-16.26 wished=4.50 severity=2.61, time=5.70.
(pid=31377) Warning: Vehicle 'human_14_0' performs emergency braking with decel=-12.34 wished=4.50 severity=1.74, time=7.00.
(pid=31377) Warning: Vehicle 'human_13_0' performs emergency braking with decel=-10.40 wished=4.50 severity=1.31, time=7.20.
(pid=31382) /home/andrei/workspace/flow_nemodrive/flow/core/kernel/vehicle/traci.py:951: UserWarning: API change now handles du

(pid=31382) /home/andrei/workspace/flow_nemodrive/flow/core/kernel/vehicle/traci.py:951: UserWarning: API change now handles duration as floating point seconds
(pid=31382)   veh_id, int(target_lane), 100000)
Result for PPO_LaneChangeAccelEnv1-v0_0:
  custom_metrics: {}
  date: 2019-11-29_14-49-35
  done: false
  episode_len_mean: 100.0
  episode_reward_max: 99.18842208334817
  episode_reward_mean: 65.9513328724793
  episode_reward_min: 21.821240417772778
  episodes_this_iter: 20
  episodes_total: 780
  experiment_id: cae98feeaba94a4fb99e19849f571f9d
  hostname: andrei
  info:
    grad_time_ms: 16579.753
    learner:
      default_policy:
        cur_kl_coeff: 0.11249999701976776
        cur_lr: 4.999999873689376e-05
        entropy: 1.4991579055786133
        entropy_coeff: 0.0
        kl: 0.020995287224650383
        policy_loss: -0.020937873050570488
        total_loss: 322.9857482910156
        vf_explained_var: -1.1920928955078125e-07
        vf_loss: 323.0044250488281
    load_tim

(pid=31376) /home/andrei/workspace/flow_nemodrive/flow/core/kernel/vehicle/traci.py:951: UserWarning: API change now handles duration as floating point seconds
(pid=31376)   veh_id, int(target_lane), 100000)
(pid=31377) /home/andrei/workspace/flow_nemodrive/flow/core/kernel/vehicle/traci.py:951: UserWarning: API change now handles duration as floating point seconds
(pid=31377)   veh_id, int(target_lane), 100000)
(pid=31377) Warning: Vehicle 'human_13_0' performs emergency braking with decel=-25.77 wished=4.50 severity=4.73, time=3.40.
(pid=31381) Warning: Vehicle 'human_14_0' performs emergency braking with decel=-32.48 wished=4.50 severity=6.22, time=3.60.
(pid=31381) Warning: Vehicle 'human_14_0' performs emergency braking with decel=-26.95 wished=4.50 severity=4.99, time=6.40.
(pid=31381) Warning: Vehicle 'human_0_0' performs emergency braking with decel=-10.11 wished=4.50 severity=1.25, time=8.40.
(pid=31375) Warning: Vehicle 'human_0_0' performs emergency braking with decel=-11.04

(pid=31381) Warning: Vehicle 'human_14_0' performs emergency braking with decel=-21.97 wished=4.50 severity=3.88, time=7.20.
(pid=31382) Warning: Vehicle 'human_9_0' performs emergency braking with decel=-49.50 wished=4.50 severity=10.00, time=7.50.
(pid=31381) Warning: Vehicle 'human_14_0' performs emergency braking with decel=-12.33 wished=4.50 severity=1.74, time=9.00.
Result for PPO_LaneChangeAccelEnv1-v0_0:
  custom_metrics: {}
  date: 2019-11-29_14-56-22
  done: false
  episode_len_mean: 100.0
  episode_reward_max: 100.53493608293832
  episode_reward_mean: 69.13951650305646
  episode_reward_min: 18.705916384815175
  episodes_this_iter: 20
  episodes_total: 840
  experiment_id: cae98feeaba94a4fb99e19849f571f9d
  hostname: andrei
  info:
    grad_time_ms: 17218.555
    learner:
      default_policy:
        cur_kl_coeff: 0.16875000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 1.519315481185913
        entropy_coeff: 0.0
        kl: 0.02827766351401806
        pol